In [3]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd

# from src.data_handler.dataloader import load_citations
from get_clean_text import load_cs_papers

In [2]:
papers = load_cs_papers("", "clean_abbv", run_preprocessor=True, to_exclude=["case-fold", "punct-removal", "tokenize"])

Loading cs papers
cs papers loaded
Executing clean
Done
Executing abbv


  0%|          | 0/604853 [00:00<?, ?it/s]c:\Users\Stefa\Downloads\ArXiv-Dataset-T14\src\scispacy\abbreviation.py:260: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
100%|██████████| 604853/604853 [16:31<00:00, 610.05it/s]  


Done


In [4]:
papers = load_cs_papers("clean_abbv")

Loading cs papers
cs papers loaded


In [6]:
papers = load_cs_papers("clean_abbv", "clean_abbv_casefold_punct", run_preprocessor=True, to_exclude=["clean", "abbv", "tokenize"])

Loading cs papers
cs papers loaded
Executing case-fold
Done
Executing punct-removal
Done


In [2]:
papers = load_cs_papers("clean_abbv_casefold_punct", "clean_abbv_casefold_punct_spacytokenizer", run_preprocessor=True, to_include=["tokenize"], no_stop=True)

Loading cs papers
cs papers loaded
Executing tokenize


100%|██████████| 604853/604853 [09:13<00:00, 1093.25it/s]


Done


In [4]:
papers = load_cs_papers("clean_abbv_casefold_punct")

Loading cs papers
cs papers loaded


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
vec = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x:x)

In [9]:
abstract_vec = vec.fit_transform(papers['abstract'])

c:\Users\Stefa\anaconda3\envs\text-mining\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
abstract_vec

<604853x666828 sparse matrix of type '<class 'numpy.float64'>'
	with 60857131 stored elements in Compressed Sparse Row format>

In [ ]:
abstract_vec

<604853x666828 sparse matrix of type '<class 'numpy.float64'>'
	with 60857131 stored elements in Compressed Sparse Row format>

In [60]:
# from transformers.modeling_bert import BertModel, BertForMaskedLM
from transformers import BertTokenizerFast, BertModel

In [61]:

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [62]:
import numpy as np
from tqdm import tqdm

In [36]:
abstracts = papers.abstract.tolist()

In [101]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [102]:
batch_size = 64

batches = int(np.ceil(len(abstracts) / batch_size))
batches

tokens = []

for i in tqdm(range(20)):
    abstracts_ = abstracts[i * batch_size: (i+1) * batch_size]
    encoded_input = tokenizer(abstracts_, return_tensors='pt', padding="max_length", truncation=True, max_length=512)
    tokens.append(encoded_input)

100%|██████████| 20/20 [00:00<00:00, 26.89it/s]


In [105]:
tokens[0]['input_ids']

tensor([[ 101, 2057, 6235,  ...,    0,    0,    0],
        [ 101, 1999, 1037,  ...,    0,    0,    0],
        [ 101, 1996, 9414,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 2465,  ...,    0,    0,    0],
        [ 101, 2592, 8346,  ...,    0,    0,    0],
        [ 101, 1996, 2591,  ...,    0,    0,    0]])

In [106]:
outputs = []
for token in tqdm(tokens):
    with torch.no_grad():
        output = model(**token)
    outputs.append(output)

 15%|█▌        | 3/20 [07:29<42:28, 149.90s/it]


KeyboardInterrupt: 

In [107]:
outputs = [output.last_hidden_state.numpy() for output in outputs]

In [117]:
global_vecs = [output[:,0,:] for output in outputs]
global_vecs = np.concatenate(global_vecs)
global_vecs

array([[-0.5287565 , -0.01068921, -0.499239  , ..., -0.41446254,
         0.02744966,  0.30988243],
       [-0.5717407 , -0.40985718, -0.27441454, ..., -0.23279467,
         0.14650892,  0.52955323],
       [-0.69186205, -0.16229591,  0.07731377, ..., -0.29492173,
        -0.20344765,  0.6825587 ],
       ...,
       [-0.83634704, -0.20692849, -0.01469363, ..., -0.47055644,
        -0.06038333,  0.32870397],
       [-0.59632474, -0.42031935, -0.01748319, ..., -0.1508324 ,
         0.07424235,  0.49122122],
       [-0.6390631 , -0.18198538, -0.25880927, ..., -0.77206844,
         0.11244365, -0.15346798]], dtype=float32)

In [138]:
from src.utils import cosine_sim, top_k_retrieval
import src.utils

In [146]:
%load_ext autoreload
%autoreload 2

In [153]:
papers.iloc[0]

id                                                     0704.0002
title                   Sparsity-certifying Graph Decompositions
abstract       we describe a new algorithm the [eqn_latex]peb...
authors                          Ileana Streinu and Louis Theran
journal-ref                                                  NaN
license        http://arxiv.org/licenses/nonexclusive-distrib...
Name: 0, dtype: object

In [171]:
i = 5
print(papers.iloc[i].title, papers.iloc[i].id)
print(sims[i][:,1])
papers.iloc[sims[i][:,0]]

Real Options for Project Schedules (ROPS) 0704.0090
[0.87688315 0.86930764 0.85959297 0.8586812  0.85861486]


,id,title,abstract,authors,journal-ref,license
32,0704.0860,Availability assessment of SunOS/Solaris Unix ...,this paper presents a measurementbased availab...,"Cristina Simache (LAAS), Mohamed Kaaniche (LAAS)",Proc. 2005 IEEE Pacific Rim International Symp...,NaN
81,0704.1925,Blind Identification of Distributed Antenna Sy...,in spatially distributed multiuser antenna sys...,"Yuanning Yu, Athina P. Petropulu and H. Vincen...",NaN,NaN
115,0704.2725,Exploiting Heavy Tails in Training Times of Mu...,the random initialization of weights of a mult...,Manuel Cebrian and Ivan Cantador,NaN,NaN
76,0704.1833,Analysis of the 802.11e Enhanced Distributed C...,the ieee e standard revises the medium access ...,"Inanc Inan, Feyza Keceli, Ender Ayanoglu",NaN,NaN
97,0704.2383,Power control and receiver design for energy e...,this paper is focused on the crosslayer design...,"Stefano Buzzi, Valeria Massaro, and H. Vincent...",NaN,NaN


In [155]:
sims = src.utils.top_k_retrieval(cosine_sim(global_vecs), top_k=5, return_score=True)

In [ ]:
def search_vec(text, top_k=1, return_scores=True, paper_subset=None, preprocessor=None, tokenizer=None, model=None):
    paper_subset_vecs = paper_subset
    if preprocessor is not None:
        text = preprocessor(text)
    search_tokens = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**search_tokens)
    search_vec = output.last_hidden_state.numpy()[:, 0, :]
    sim_papers = src.utils.top_k_retrieval(cosine_sim(search_vec, paper_subset_vecs), top_k=top_k, return_score=True)
    sim_papers_index = sim_papers[:, 0]
    sim_papers_score = sim_papers[:, 1]
    
    sim_papers = paper_subset.iloc[sim_papers_index]
    if return_scores:
        sim_papers.loc[:, "similarity_score"] = sim_papers_score

    return sim_papers

In [189]:
text = "Adam a method for stochastic optimization"

In [190]:
inp_search = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=512)

In [191]:
with torch.no_grad():
    output = model(**inp_search)

In [192]:
search_vec = output.last_hidden_state.numpy()[:,0,:]

In [195]:
sim_papers = src.utils.top_k_retrieval(cosine_sim(search_vec, global_vecs), top_k=10, return_score=True)

In [196]:
papers.iloc[sim_papers[0][:,0]]

,id,title,abstract,authors,journal-ref,license
7,0704.0108,Reducing SAT to 2-SAT,description of a polynomial time reduction of ...,Sergey Gubin,NaN,NaN
134,0704.3157,Experimenting with recursive queries in databa...,this paper considers the problem of reasoning ...,"Giorgio Terracina, Nicola Leone, Vincenzino Li...",NaN,NaN
70,0704.1756,The Invar Tensor Package,the invar package is introduced a fast manipul...,"Jose M. Martin-Garcia, Renato Portugal, Leon R...",Comp. Phys. Commun. 177 (2007) 640-648,NaN
117,0704.2779,The Complexity of Simple Stochastic Games,in this paper we survey the computational time...,Jonas Dieckelmann,NaN,NaN
12,0704.0282,On Punctured Pragmatic Space-Time Codes in Blo...,this paper considers the use of punctured conv...,"Samuele Bandi, Luca Stabellini, Andrea Conti a...",NaN,NaN
93,0704.2351,Parallel computation of the rank of large spar...,this paper deals with the computation of the r...,"Jean-Guillaume Dumas (LMC - IMAG), Philippe El...",NaN,NaN
123,0704.2857,Modern Coding Theory: The Statistical Mechanic...,these are the notes for a set of lectures deli...,Andrea Montanari and Rudiger Urbanke,NaN,NaN
6,0704.0098,Sparsely-spread CDMA - a statistical mechanics...,sparse code division multiple access a variati...,"Jack Raymond, David Saad",J. Phys. A: Math. Theor. 40 No 41 (12 October ...,NaN
108,0704.2596,Computing Extensions of Linear Codes,this paper deals with the problem of increasin...,Markus Grassl,Proceedings 2007 IEEE International Symposium ...,NaN
180,0704.3780,Stochastic Optimization Algorithms,when looking for a solution deterministic meth...,"Pierre Collet, Jean-Philippe Rennard","Rennard, J.-P., Handbook of Research on Nature...",NaN


In [48]:
abstracts[0:64]

['we describe a new algorithm the [eqn_latex]pebble game with colors and use it obtain a characterization of the family of [eqn_latex]sparse graphs and algorithmic solutions to a family of problems concerning tree decompositions of graphs special instances of sparse graphs appear in rigidity theory and have received increased attention in recent years in particular our colored pebbles generalize and strengthen the previous results of lee and streinu and give a new proof of the tuttenashwilliams characterization of arboricity we also present a new decomposition that certifies sparsity based on the [eqn_latex]pebble game with colors our work also exposes connections between pebble game algorithms and previous sparse graph algorithms by gabow gabow and westermann and hendrickson',
 'in a quantum mechanical model diosi feldmann and kosloff arrived at a conjecture stating that the limit of the entropy of certain mixtures is the relative entropy as system size goes to infinity the conjecture

In [53]:
tokenizer(abstracts[0:64], return_tensors='pt', padding=True)['input_ids']

torch.Size([64, 317])

In [45]:
len(abstracts[9449 * 64:9450*64])

64

In [34]:
np.ceil(3.5)

4.0

In [30]:
bert_tokens = tokenizer(papers.abstract.tolist(), return_tensors='pt')

KeyboardInterrupt: 

In [11]:
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

In [12]:
svd = TruncatedSVD(n_components=1024)
reduced_vec = svd.fit_transform(abstract_vec)

KeyboardInterrupt: 

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(600_000 // 50 )

In [ ]:
kmeans.fit_transform(reduced_vec)